# ABS Inflation multi-measure

## Python set-up

In [1]:
# system imports
from typing import Callable, Sequence, cast
from functools import cache

# analytic imports
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import pandas as pd
from pandas import DataFrame
import numpy as np
import readabs as ra
from readabs import metacol as mc
import mgplot as mg

# local imports
from abs_helper import ANNUAL_CPI_TARGET_RANGE

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# save charts in this notebook
CHART_DIR = "./CHARTS/Inflation/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()

# some plotting constants
LFOOTER = "Australia. Orig = Original series. SA = Seasonally adjusted series. "

# display charts in this notebook
SHOW = False

## NEW -- with a different data capture mechanism

In [3]:
stype_codes = {
    "O": "Original",
    "S": "Seasonally Adjusted",
    "SA": "Seasonally Adjusted",
    "T": "Trend",
}

In [4]:
@cache
def get_table(cat, table) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Get ABS data table and metadata for a given ABS catalogue-id and table-id."""

    dictionary, meta = ra.read_abs_cat(cat, single_excel_only=table)
    data = dictionary[table]
    return data, meta

In [5]:
@cache
def load_series(input_tuple: tuple[str, str, str, str, str]) -> DataFrame:
    """Load ABS series from readabs and return as DataFrame."""
    cat, table, did, stype, unit = input_tuple
    stype = stype if stype not in stype_codes else stype_codes[stype]
    data, meta = get_table(cat, table)

    ret = pd.DataFrame()
    for u in ["index", "change"]:
        if not unit or u in unit.lower():

            typology: str = "Index" if u == "index" else "Previous Year"
            selector = {
                did: mc.did,
                typology: mc.did,
                stype: mc.stype,
            }
            _table, series_id, units = ra.find_abs_id(meta, selector)
            ret[u] = data[series_id]
    return ret

In [6]:
def old_quarterly() -> dict[str, DataFrame]:
    """Load quarterly CPI series from ABS."""

    wanted = {
        # name: (cat, table, did, stype, unit)
        "Qrtly Headline CPI (Orig)" :
            ("6401.0", "640106", "All groups CPI ;", "O", ""),
        "Qrtly Headline CPI (SA)" :
            ("6401.0", "640106", "All groups CPI, seasonally adjusted", "S", ""),
        "Qrtly Trimmed Mean CPI (SA)" :
            ("6401.0", "640106", "Trimmed Mean", "S", ""),
        "Qrtly Weighted Median CPI (SA)" :
            ("6401.0", "640106", "Weighted Median", "S", ""),
        "Qrtly Goods Component CPI (Orig)":
            ("6401.0", "640106", "All groups, goods component", "O", ""),
        "Qrtly Services Component CPI (Orig)":
            ("6401.0", "640106", "All groups, services component", "O", ""),
        "Qrtly Tradables CPI (Orig)":
            ("6401.0", "640106", "Tradables", "O", ""),
        "Qrtly Non-tradables CPI (Orig)":
            ("6401.0", "640106", "Non-tradables", "O", ""),

    }

    box = {}
    for (w, t) in wanted.items():
        df = load_series(t)
        box[w] = df

    return box


#box_test = old_quarterly()

In [7]:
def annualise_quarterly_plot() -> None:
    """Plot annualised inflation indicators from quarterly data only."""

    # --- constants ---
    start = pd.Period("2019Q4", freq="Q")

    # get data - return to quarterly
    box_of_data = old_quarterly()
    pairs = {
        "CPI Goods and Services": (
            "Qrtly Goods Component CPI (Orig)",
            "Qrtly Services Component CPI (Orig)",
        ),
        "CPI Tradables": (
            "Qrtly Tradables CPI (Orig)",
            "Qrtly Non-tradables CPI (Orig)",
        ),
        "CPI Headline and Core": (
            "Qrtly Headline CPI (SA)",
            "Qrtly Trimmed Mean CPI (SA)",
        ),
        "CPI Goods and Tradables": (
            "Qrtly Goods Component CPI (Orig)",
            "Qrtly Tradables CPI (Orig)",
        ),
        "CPI Services and Non-tradables": (
            "Qrtly Services Component CPI (Orig)",
            "Qrtly Non-tradables CPI (Orig)",
        ),
    }

    qtrs_per_year = 4
    for quarters in range(1, 5):
        for label, pair in pairs.items():
            p0, p1 = box_of_data[pair[0]]["index"], box_of_data[pair[1]]["index"]
            dataset = pd.concat([p0, p1], axis=1)
            dataset.columns = [pair[0], pair[1]]

            pct_d = ra.percent_change(dataset, n_periods=quarters)
            stem = ""
            if quarters != 4:
                pct_d = ra.annualise_percentages(
                    data=pct_d, periods_per_year=qtrs_per_year / quarters
                )
                stem = f" - {quarters} quarters annualised"
                stem = stem.replace("quarters", "quarter") if quarters == 1 else stem

            mg.line_plot_finalise(
                pct_d[start:],
                title=f"{label} Growth{stem}",
                ylabel="Per cent (through the year)",
                axhspan=ANNUAL_CPI_TARGET_RANGE,
                y0=True,
                rfooter="ABS: 6401.0",
                legend=True,
                lfooter=LFOOTER,
                annotate=True,
                width=[1, 2],
                show=SHOW,
            )


annualise_quarterly_plot()

## OLD -- What we did previously

### Download key ABS data

In [8]:
def download_abs(
    wanted: dict[str, tuple[str, str, str, str]]
) -> tuple[pd.DataFrame, pd.DataFrame, dict[str, str], str]:
    """Capture data from ABS using a wanted dictionary. If ULC or
    IPD is in the abbreviation, then the data is converted to a
    4-quarter percentage change. If the frequency is quarterly, then
    Arguments:
    - wanted: a dictionary of {series_id: (category_id, seo, title, abrev)}
    Returns a 4-tuple of:
    - DataFrame: the data (with a monthly PeriodIndex)
    - DataFrame: the metadata
    - dict: the abbreviations (from title -> abrev in wanted)
    - str: the sources string."""

    data, meta, abbr, sources = {}, {}, {}, []
    for series_id, (category_id, seo, title, abrev) in wanted.items():
        d, m = ra.read_abs_series(category_id, series_id, single_excel_only=seo)
        series = d[series_id]
        periodocity = 4 if "Q" in m[mc.freq][series_id] else 12
        if any(x in abrev for x in ["IPD", "ULC"]):
            series = series.pct_change(periodocity) * 100
        if periodocity == 4:
            series = ra.qtly_to_monthly(series, interpolate=False)
        data[title] = series
        meta[title] = m.loc[series_id]
        abbr[title] = abrev
        sources.append(category_id)

    return (
        pd.DataFrame(data),
        pd.DataFrame(meta).T,
        abbr,
        f"ABS: {', '.join(sorted(set(sources)))}",
    )

### Multi-indicator plot

In [9]:
def plot_pairs(
    function: Callable,
    pairs: dict[str, tuple[str, str]],
    start: str = "2019-12-01",
    title_stem: str = "Monthly vs Quarterly",
) -> None:
    """Plot pairs of inflation indicators."""

    data, _meta, _abbr, source = function()
    data = data.loc[lambda x: x.index >= start]
    for label, pair in pairs.items():
        d = data[list(pair)]
        mg.line_plot_finalise(
            d,
            title=f"{label}: {title_stem}",
            ylabel="Per cent (through the year)",
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            y0=True,
            rfooter=source,
            legend=True,
            lfooter=LFOOTER,
            width=[1, 2],
            show=SHOW,
            dropna=True,
        )


In [10]:
IPD = "5206005_Expenditure_Implicit_Price_Deflators"
WANTED = {
        # "Series ID": ("Category ID", "single-excel-only", "Short Series Title", "abbreviation"),
        # note: if "IPD" or "ULC" is in the abbreviation, the series is converted to a annual % change
        "A128478318V": ("6484.0", "648401", "Monthly CPI (Orig)", "MCPI"),
        "A128481588C": ("6484.0", "648401", "Monthly CPI (SA)", "MCPISA"),
        "A130184497K": (
            "6484.0",
            "648401",
            "Monthly Trimmed Mean CPI (Orig)",
            "MCPITM",
        ),
        "A2325847F": ("6401.0", "640106", "Qtly CPI (Orig)", "CPI"),
        "A3604508K": ("6401.0", "640106", "Qtly CPI (SA)", "CPISA"),
        "A3604511X": ("6401.0", "640106", "Qtly Trimmed Mean CPI (SA)", "CPITM"),
        "A3604505C": ("6401.0", "640106", "Qtly Weighted Median CPI (SA)", "CPIWM"),
        "A2314867K": ("6427.0", "642701", "Producer Price Index (Orig)", "PPI"),
        "A83895396W": (
            "6345.0",
            "634501",
            "Wage Price Index (All sectors) (SA)",
            "WPI",
        ),
        "A2303940R": (
            "5206.0",
            IPD,
            "Household implicit price deflator (SA)",
            "HHIPD",
        ),
        "A2303727C": ("5206.0", IPD, "GNE implicit price deflator (SA)", "GNEIPD"),
        #"A2433068T": (
        #    "5206.0",
        #    "5206042_Unit_Labour_Costs",
        #    "Nominal unit labour costs (SA)",
        #    "ULC",
        #),
    }

REMOVE_FOR_UNDERLYING = (
    # these headline or monthly indicators are more noisy
    #"A2433068T", 
    "A3604508K", "A2325847F", "A130184497K", "A128481588C", "A128478318V"
)

In [11]:
@cache
def get_headline_data() -> tuple[pd.DataFrame, pd.DataFrame, dict[str, str], str]:
    """Get a dictionary of data items from the ABS."""

    return download_abs(WANTED)

In [12]:
def plot_multi() -> None:
    """Plot multiple inflation indicators, for a number of different
    start dates, with different styles and markers."""


    for dataset in ("full", "reduced"):

        data, _meta, abbr, source = get_headline_data()
        if dataset == "reduced":
            data = data.drop(columns=
                [WANTED[k][2] for k in REMOVE_FOR_UNDERLYING]
            )
            cats = set(
                WANTED[c][0] for c in WANTED if c not in REMOVE_FOR_UNDERLYING)
            source = f"ABS: {', '.join(cats)}"

        last = pd.Series(data.ffill().iloc[-1]).rename(index=abbr)
        last_str = ", ".join([f"{k}={v:0.1f}" for k, v in last.items()])

        mixed_style = ["solid", "dashed", "dotted"] * 5  # not using 'dashdot',
        marker_set = list("ov^<>8sp*HhdPX")

        for start, tag, style, marker in zip(
            (None, "2019-12-01", "2022-12-01", "2024-09-01"),  # starts
            (
                "",
                " (since COVID)",
                " (from Dec-2022)",
                " (latest)",
            ),  # tags
            ("-", mixed_style, mixed_style, mixed_style),  # styles
            (None, marker_set, marker_set, marker_set),  # markers
        ):
            starter = pd.Period(start, freq="M") if start else None
            d = data.loc[[i for i in data.index if i >= starter]] if starter else data
            mg.line_plot_finalise(
                d,
                title=f"Inflation Indicators{tag}",
                ylabel="Per cent (through the year)",
                axhspan=ANNUAL_CPI_TARGET_RANGE,
                y0=True,
                rfooter=source,
                lheader=last_str,
                legend={
                    "loc": "best",
                    "fontsize": "xx-small",
                    "ncol": 2,
                },
                tag=dataset,
                lfooter=LFOOTER,
                marker=marker,
                markersize=4,
                style=style,
                dropna=True,
                show=SHOW,
            )


plot_multi()

### Quarterly/Monthly CPI pairs

In [13]:


headline_pairs = {
    # "label": ("quarterly series", "monthly series")
    "Headline CPI (Orig)": ("Qtly CPI (Orig)", "Monthly CPI (Orig)"),
    "Headline CPI (SA)": ("Qtly CPI (SA)", "Monthly CPI (SA)"),
    "Trimmed Mean CPI": (
        "Qtly Trimmed Mean CPI (SA)",
        "Monthly Trimmed Mean CPI (Orig)",
    ),
    "Weighted Median CPI": (
        "Qtly Weighted Median CPI (SA)",
        "Monthly Trimmed Mean CPI (Orig)",
    ),
}

plot_pairs(get_headline_data, headline_pairs)

In [14]:
@cache
def get_analytic_data() -> tuple[pd.DataFrame, pd.DataFrame, dict[str, str], str]:
    """Get a dictionary of data items from the ABS."""

    wanted = {
        # "Series ID": ("Category ID", "single-excel-only", "Short Series Title", "abbreviation"),
        "A128483462J": ("6484.0", "648401", "Monthly CPI Goods Component (Orig)", ""),
        "A128481640A": (
            "6484.0",
            "648401",
            "Monthly CPI Services Component (Orig)",
            "",
        ),
        "A128476506T": ("6484.0", "648401", "Monthly CPI Tradables (Orig)", ""),
        "A128480134T": ("6484.0", "648401", "Monthly CPI Non-tradables (Orig)", ""),
        "A130184498L": ("6484.0", "648401", "Monthly CPI Excluding Volatile (SA)", ""),
        "A2330617V": ("6401.0", "640106", "Qtly CPI Goods Component (Orig)", ""),
        "A2330707X": ("6401.0", "640106", "Qtly CPI Services Component (Orig)", ""),
        "A2330527R": ("6401.0", "640106", "Qtly CPI Tradables (Orig)", ""),
        "A2330572A": ("6401.0", "640106", "Qtly CPI Non-tradables (Orig)", ""),
        "A2330842R": ("6401.0", "640106", "Qtly CPI Excluding Volatile (Orig)", ""),
    }
    return download_abs(wanted)

In [15]:
analytic_pairs = {
    # "label": ("quarterly series", "monthly series")
    "CPI Goods Component": (
        "Qtly CPI Goods Component (Orig)",
        "Monthly CPI Goods Component (Orig)",
    ),
    "CPI Services": (
        "Qtly CPI Services Component (Orig)",
        "Monthly CPI Services Component (Orig)",
    ),
    "CPI Tradables": ("Qtly CPI Tradables (Orig)", "Monthly CPI Tradables (Orig)"),
    "CPI Non-tradables": (
        "Qtly CPI Non-tradables (Orig)",
        "Monthly CPI Non-tradables (Orig)",
    ),
    "CPI Excluding Volatile": (
        "Qtly CPI Excluding Volatile (Orig)",
        "Monthly CPI Excluding Volatile (SA)",
    ),
}

plot_pairs(get_analytic_data, analytic_pairs)

## Annualised monthly/quarterly

***But note***: a whole host of problems annualising the monthly series.
And the data does not exist to annualise the monthly trimmed mean CPI.

In [16]:
@cache
def get_index_data() -> pd.DataFrame:
    """Get a dictionary of data items from the ABS."""

    # NOTE: these are quarterly index series only (not percentage changes)
    wanted = {
        # "Series ID": ("Category ID", "single-excel-only", "Short Series Title", "abbreviation", ""),
        "A2325846C": ("6401.0", "640106", "Qtly CPI (Orig)", ""),
        "A3604506F": ("6401.0", "640106", "Quartely CPI (SA)", ""),
        "A2330841L": ("6401.0", "640106", "Qtly CPI Excluding Volatile (Orig)", ""),
        "A2330616T": ("6401.0", "640106", "Qtly CPI Goods Component (Orig)", ""),
        "A2330706W": ("6401.0", "640106", "Qtly CPI Services Component (Orig)", ""),
        "A128478317T": ("6484.0", "648401", "Monthly CPI (Orig)", ""),
        "A128481587A": ("6484.0", "648401", "Monthly CPI (SA)", ""),
        "A128473239F": (
            "6484.0",
            "648401",
            "Monthly CPI Excluding Volatile (Orig)",
            "",
        ),
        "A128483461F": ("6484.0", "648401", "Monthly CPI Goods Component (Orig)", ""),
        "A128481639T": (
            "6484.0",
            "648401",
            "Monthly CPI Services Component (Orig)",
            "",
        ),
    }

    return download_abs(wanted)[0]

In [17]:
def annualise_plot() -> None:
    """Plot annualised inflation indicators."""

    # pairs we want to plot
    data: DataFrame = get_analytic_data()[0]
    pairs = {
        # "label": ("quarterly series", "monthly series")
        "Headline CPI": ("Qtly CPI (Orig)", "Monthly CPI (Orig)"),
        "Headline CPI (SA)": ("Quartely CPI (SA)", "Monthly CPI (SA)"),
        "CPI Excluding Volatile": (
            "Qtly CPI Excluding Volatile (Orig)",
            "Monthly CPI Excluding Volatile (Orig)",
        ),
        "CPI Goods Component": (
            "Qtly CPI Goods Component (Orig)",
            "Monthly CPI Goods Component (Orig)",
        ),
        "CPI Services Component": (
            "Qtly CPI Services Component (Orig)",
            "Monthly CPI Services Component (Orig)",
        ),
    }

    # calculate annualised percentage changes and plot
    months_in_year = 12
    for months in 3, 6:
        pct_d = ra.percent_change(data, n_periods=months)
        pct_d = ra.annualise_percentages(data=pct_d, periods_per_year=months_in_year / months)

        def fake_capture(dataset=pct_d):
            return dataset, DataFrame(), "ABS: 6401.0, 6484.0", ""

        plot_pairs(
            fake_capture,
            pairs,
            title_stem=f"{months} months annualised",
        )


#annualise_plot()

## Phillips Curves

In [18]:
def xy_plot(
    frame: DataFrame,
    line_label: str,
    point_labels: Sequence[int] | None = None,
    ax = None,
    **kwargs,
) -> Axes:
    """ "Plot and label the heart of the curve.

    Arguments:
    frame is a two column DataFrame, first col is x vales, 2nd col is y values.
    line_label is the label for the line.
    point_labels is the points to label.

    Returns:
    An Axes object."""

    width = kwargs.pop("width", 2)
    color = kwargs.pop("color", "black")
    
    ax = frame.plot(
        x=frame.columns[0],
        y=frame.columns[1],
        lw=width,
        color=color,
        label=line_label if line_label else "_nolegend_",
        ax=ax,
    )
    point_labels = [] if point_labels is None else point_labels
    for n in point_labels:
        # Label the start and end
        ax.text(
            frame[frame.columns[0]].iloc[n],
            frame[frame.columns[1]].iloc[n],
            f"{frame.index[n]} ",
            fontsize="x-small",
            ha="right",
        )
    return ax

In [19]:
def add_regression(
    ax,
    frame: DataFrame,
    line_label: str,
    **kwargs: int | str,
) -> None:
    """Fit a polynomial regression line to the data."""

    color: str = str(kwargs.get("color", "darkred"))
    degree: int = int(kwargs.get("degree", 1))
    linestyle: str = str(kwargs.get("linestyle", "--"))
    width: float = float(kwargs.get("width", 0.75))

    model = np.poly1d(
        np.polyfit(frame[frame.columns[0]], frame[frame.columns[1]], degree)
    )
    polyline = np.linspace(
        frame[frame.columns[0]].min(), frame[frame.columns[0]].max(), 50
    )
    ax.plot(
        polyline,
        model(polyline),
        color=color,
        linestyle=linestyle,
        lw=width,
        label=line_label if line_label else "_nolegend_",
    )

In [20]:
def tm()-> pd.Series:

    cpi, _ = ra.read_abs_series("6401.0", "A3604509L", single_excel_only="640106")
    tm_cpi = cpi["A3604509L"].pct_change(periods=4) * 100

    return tm_cpi

def uer() -> pd.Series:

    lfs, _ = ra.read_abs_series(
        "6202.0", ["A84423043C", "A84423047L"], single_excel_only="6202001"
    )
    ue_rate = ra.monthly_to_qtly(100 - (lfs["A84423043C"] / lfs["A84423047L"] * 100))

    return ue_rate



In [21]:
def phillips_curve() -> None:
    """Produce a Phillips Curve chart."""

    # --- Organise the data
    ue_rate = uer()
    tm_cpi = tm()

    # household implicit price deflator (seasonally adjusted)
    ipd, _ = ra.read_abs_series(
        "5206.0",
        "A2303940R",
        single_excel_only="5206005_Expenditure_Implicit_Price_Deflators",
    )
    hheipd = ipd["A2303940R"].pct_change(periods=4) * 100

    inflation = {
        "Trimmed Mean CPI": tm_cpi,
        "Household Expenditure IPD": hheipd,
    }

    start = "2021Q1"  # "2009Q4"  #
    for label, series in inflation.items():

        frame = pd.DataFrame({label: series, "_Unemployment Rate": ue_rate})
        last: float = (
            cast(float, frame.iloc[-1, 1]) if frame.iloc[-1].isna().any() else 0.0
        )
        last_date = frame.index[-1]
        frame = frame.loc[
            lambda x: x.index >= start
        ].dropna()  # drop the last row if it has a NaN

        # --- Plot the data
        ax = xy_plot(frame, "Phillips curve", point_labels=(0, -1))
        add_regression(ax, frame, "Stylised Phillips curve", degree=3)

        ax.axvline(
            2.5, color="darkblue", linestyle=":", lw=0.75, label="2.5% Inflation target"
        )
        if last > 0.0:
            ax.axhline(
                last,
                color="darkgreen",
                linestyle="-.",
                lw=0.75,
                label=f"Unemployment rate {last_date}",
            )
        mg.finalise_plot(
            ax,
            title=f"Phillips Curve: {label} vs Unemployment",
            ylabel="Unemployment Rate (%)",
            xlabel=f"{label} Annual Growth Rate (%)",
            lfooter="Australia, Seasonally adjusted. Unemployment rate is quarterly mean. ",
            rfooter="Source: ABS",
            legend=True,
            show=SHOW,
        )


phillips_curve()

In [22]:
def stylised_phillips():
    """Produce a Stylised Phillips Curve chart for different periods."""
    
    ue_rate = uer()
    tm_cpi = tm()

    period_sets = [
        [
            list(pd.period_range(pd.Period("2021Q1", freq="Q"), tm_cpi.index[-1])),
            "brown",
            3,
        ],
        [
            list(pd.period_range(pd.Period("2010Q1", freq="Q"), pd.Period("2019Q4", freq="Q"))),
            "blue",
            1,
        ],
        [
            list(pd.period_range(pd.Period("2003Q1", freq="Q"), pd.Period("2008Q4", freq="Q"))),
            "orange",
            3,
        ],
        [
            list(pd.period_range(pd.Period("1998Q1", freq="Q"), pd.Period("2002Q4", freq="Q"))),
            "skyblue",
            1,
        ],
    ]
    
    _, ax = plt.subplots()
    
    for start_stop, color, degree in period_sets:
        assert len(start_stop) >= 2, f"start_stop len={len(start_stop)} must have start and end"
        label = f"{start_stop[0]}-{start_stop[-1]}"
        
        frame = pd.DataFrame({
            "Trimmed Mean CPI": tm_cpi,
            "Unemployment Rate": ue_rate,
        })
        frame = frame.loc[start_stop[0]:start_stop[-1]].dropna()

        xy_plot(
            frame,
            line_label="",
            point_labels=[],
            ax=ax,
            width=0.5,
            color=color,
        )
        
        add_regression(
            ax,
            frame,
            label,
            degree=degree,
            color=color,
            linestyle="-",
            width=3,
        )

    axvline = {
        "x": 2.5,
        "color": "darkblue",
        "linestyle": ":",
        "lw": 0.75,
        "label": "2.5% Inflation target",
    }
    
    mg.finalise_plot(
        ax,
        title="Stylised Phillips Curves: Trimmed Mean CPI vs Unemployment",
        ylabel="Unemployment Rate (%)",
        xlabel="Trimmed Mean CPI Annual Growth Rate (%)",
        lfooter="Australia, Seasonally adjusted. Unemployment rate is quarterly mean.",
        rfooter="Source: ABS",
        axvline=axvline,
        legend=True,
        show=SHOW,
    )


stylised_phillips()

## Beveridge curve

In [23]:
def beveridge_curve() -> None:
    """Plot a Beveridge curve."""

    # --- Collect data for the next chart
    lfs, _ = ra.read_abs_series(
        "6202.0", ["A84423043C", "A84423047L"], single_excel_only="6202001"
    )
    ue_rate_q = ra.monthly_to_qtly(
        100 - (lfs["A84423043C"] / lfs["A84423047L"] * 100), q_ending="NOV"
    )
    la, _ = ra.read_abs_series("6354.0", "A590698F", single_excel_only="6354001")
    vr = la["A590698F"] / ra.monthly_to_qtly(lfs["A84423047L"], q_ending="NOV") * 100
    frame = pd.DataFrame({"Unemployment Rate": ue_rate_q, "Job Vacancy Rate": vr})
    start = "2010Q2"
    frame = frame.loc[lambda x: x.index >= start].dropna()

    # --- Plot the next chart
    labels = [0, -1, -5, -9, -13, -17]  # Will need to update from time to time
    ax = xy_plot(frame, "Beveridge curve", point_labels=labels)
    points = frame.loc[lambda x: x.index < "2020Q2"].index.union(
        frame.loc[lambda x: x.index >= "2022Q4"].index
    )
    f = frame.loc[points]
    add_regression(ax, f, "Pre-COVID Beveridge curve (stylised)", degree=2)
    points = frame.loc[lambda x: (x.index >= "2020Q3") & (x.index <= "2022Q2")].index
    f = frame.loc[points]
    add_regression(
        ax,
        f,
        "Post-COVID Beveridge curve (stylised)",
        degree=2,
        color="darkgreen",
        linestyle="-.",
    )
    mg.finalise_plot(
        ax,
        title="Beveridge Curve: Unemployment Rate vs Job Vacancy Rate",
        ylabel="Job Vacancy Rate (%)",
        xlabel="Unemployment Rate (%)",
        lfooter="Australia, Seasonally adjusted. Unemployment rate is quarterly mean "
        + "(quarters ending Feb, May, Aug, Nov).",
        rfooter="Source: ABS 6202.0, 6354.0",
        legend=True,
        show=SHOW,
    )


beveridge_curve()

### Okun's curve

Note: this is a faily weak relationship

In [24]:
def okun():
    """Plot Okun's curve."""

    # seasonally adjusted unemployment rate - post 1978
    lfs, _ = ra.read_abs_series(
        "6202.0", ["A84423043C", "A84423047L"], single_excel_only="6202001"
    )
    ue_rate = ra.monthly_to_qtly(100 - (lfs["A84423043C"] / lfs["A84423047L"] * 100))

    # get historic unemployment - quarterly data
    old, _old_meta = ra.read_abs_series(
        cat="1364.0.15.003", series_id=["A2454521V", "A2454517C"]
    )
    uemployed, labour_force = old["A2454521V"], old["A2454517C"]
    ue_hist = ((uemployed / labour_force) * 100).dropna()

    # combine into a single quarterly unemployment rate series
    u = (
        ue_rate.reindex(
            pd.period_range(start=ue_hist.index.min(), end=ue_rate.index.max())
        )
        .sort_index()
        .pipe(lambda x: x.where(x.notnull(), ue_hist))
    )

    # GDP growth rate - seasonally adjusted chain volume measure percentage change
    na, _ = ra.read_abs_series(
        "5206.0", "A2304402X", single_excel_only="5206001_Key_Aggregates"
    )
    gdp = na["A2304402X"]
    for n, n_desc in zip((1, 4), ("Q/Q", "Y/Y")):
        du = u.diff(n).dropna()

        d_gdp = gdp.pct_change(n) * 100

        # join the two series and plot
        x, y = f"Change Unemployment Rate {n_desc}", f"GDP Growth Rate {n_desc}"
        data = pd.DataFrame({x: du, y: d_gdp}).dropna()
        ax = data.plot.scatter(x=x, y=y, s=10, alpha=0.5)

        m, b = np.polyfit(data[x].to_numpy(), data[y].to_numpy(), 1)
        xx = np.linspace(data[x].min(), data[x].max(), 2)
        yy = m * xx + b
        ax.plot(xx, yy, color="red", lw=2, label=f"Regression line (m={m:0.2f})")

        colors = [
            "grey",
            "darkorange",
            "darkorchid",
            "maroon",
            "navy",
            "green",
            "hotpink",
        ]
        styles = ["--", "-.", ":"] * 3
        for i, decade in enumerate(range(1960, 2030, 10)):
            subset = data[data.index.year // 10 == decade // 10]
            m, b = np.polyfit(subset[x].to_numpy(), subset[y].to_numpy(), 1)
            xx = np.linspace(subset[x].min(), subset[x].max(), 2)
            yy = m * xx + b
            ax.plot(
                xx,
                yy,
                color=colors[i],
                ls=styles[i],
                lw=1.5,
                label=f"Regression line for {decade}s (m={m:0.2f})",
            )

        mg.finalise_plot(
            ax,
            title=f"Okun Curve: Change in Unemployment Rate vs GDP Growth {n_desc}",
            ylabel=y + " (%)",
            xlabel=x + " (pp)",
            lfooter="Australia. Unemployment rate is quarterly mean. "
            + "In the legend, m is the slope of the regression line.",
            rfooter="ABS: 6202.0, 5206.0, 1364.0.15.003",
            legend={
                "loc": "best",
                "fontsize": "x-small",
                "ncol": 2,
            },
            x0=True,
            y0=True,
            show=SHOW,
        )

        if n == 1:
            # quarterly data is just too noisy to be useful
            continue

        # recent data
        recent_date = "2021Q3"
        recent_data = data[recent_date:]
        ax = xy_plot(
            recent_data,
            "Okun Curve",
            point_labels=(0, -1),
        )
        add_regression(
            ax,
            recent_data,
            "Post-COVID Okun curve (stylised)",
            degree=1,
            color="darkgreen",
            linestyle="-.",
        )

        mg.finalise_plot(
            ax,
            title=f"{n_desc} Okun Curve: since {recent_date}",
            ylabel=y + " (%)",
            xlabel=x + " (pp)",
            lfooter="Australia. Unemployment rate is quarterly mean. ",
            rfooter="ABS: 6202.0, 5206.0",
            legend=True,
            x0=True,
            y0=True,
            show=SHOW,
        )


okun()

### WPI - CPI Q

In [25]:
def cpi_v_wpi() -> None:
    """Compare WPI with the CPI."""

    wanted = {
        # the Actual index
        "A3604506F": ("6401.0", "640106", "Consumer Price Index (SA)"),
        "A2713849C": (
            "6345.0",
            "634501",
            "Wage Price Index (All sectors) (SA)",
        ),
    }
    base = "2019Q4"
    frame = pd.DataFrame()
    cats = []
    for series_id, (category_id, seo, title) in wanted.items():
        cats.append(category_id)
        d, _m = ra.read_abs_series(category_id, series_id, single_excel_only=seo)
        series = d[series_id]
        series = series / series.loc[base] * 100
        frame[title] = series[base:]  # type: ignore[misc]
        mg.line_plot_finalise(
            frame,
            title="Inflation and Wages",
            ylabel="Index (2019Q4=100)",
            lfooter=f"{LFOOTER}Quarterly data. ",
            rfooter=f"ABS {', '.join(cats)}",
            legend=True,
            width=2,
            show=SHOW,
        )


cpi_v_wpi()

## Finished

In [26]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-11-19 15:01:14

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.6.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

typing    : 3.10.0.0
pandas    : 2.3.3
readabs   : 0.1.4
mgplot    : 0.2.12
numpy     : 2.3.4
matplotlib: 3.10.7

Watermark: 2.5.0



In [27]:
print("Finished")

Finished
